In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
num_input = 3
num_output = 3

num_hidden1 = 16
num_hidden2 = 16

my_rmse_list = []

In [15]:

session = tf.Session()

x = tf.placeholder(tf.float32, shape=[None, num_input])

y_true = tf.placeholder(tf.float32, shape=[None, num_output], name='y_true')
y_pred = tf.placeholder(tf.float32, shape=[None, num_output])


In [16]:
data_csv = pd.read_csv('../data.csv')
X = data_csv[['in1','in2','in3']]
y = data_csv[['out1','out2','out3']]
#X, y
print (type(X), type(y))
#y

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [17]:

X_a = X.values
y_a = y.values
y_dumb = X_a*2

In [18]:

def cost_func(layer, my_y_true):
    ret = tf.sqrt(tf.reduce_mean(tf.square(layer - my_y_true)))
    
    return ret

In [19]:
def nn_model_fn(features, labels, mode):
    global my_rmse_list
    # Neural Network Structure
    # Input Layer
    input_layer = tf.reshape(features["x"],[-1, 3]) 
    
    # Hidden Layer #1
    h1 = tf.layers.dense(inputs=input_layer, 
                         units=num_hidden1,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Hidden Layer #2
    h2 = tf.layers.dense(inputs=h1, 
                         units=num_hidden2,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Output Layer
    output_layer = tf.layers.dense(inputs=h2, 
                         units=num_output,
                         use_bias=True,
                         activation=None)
    predictions = {
        "output" : output_layer
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    cost = cost_func(output_layer, labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
        
        train_op = optimizer.minimize(
                    loss=cost,
                    global_step=tf.train.get_global_step())
        #my_rmse_list += [cost.eval]
        return tf.estimator.EstimatorSpec(mode=mode, loss=cost, train_op=train_op)
    
    eval_metric_ops = {
        "error":tf.metrics.mean_squared_error(
        labels=labels, predictions=predictions["output"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=cost, eval_metric_ops=eval_metric_ops)

In [20]:
my_nn_esti = tf.estimator.Estimator(model_fn=nn_model_fn, model_dir="./sav")

tensors_to_log = {"RMSE": "output"}
rmse_list = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './sav', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000198133AB278>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    #x={"in1" : X_a[:,0], "in2":X_a[:,1], "in3":X_a[:,2]},
                    x={"x":X_a},
                    #y=y_a,
                    y=y_a,
                    batch_size=1000,
                    num_epochs=None,
                    shuffle=True)
my_nn_esti.train(
                input_fn=train_input_fn,
                steps=100000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./sav\model.ckpt.
INFO:tensorflow:loss = 0.6555980891354246, step = 1
INFO:tensorflow:global_step/sec: 136.569
INFO:tensorflow:loss = 0.2530371296024074, step = 101 (0.733 sec)
INFO:tensorflow:global_step/sec: 143.454
INFO:tensorflow:loss = 0.31283530208261295, step = 201 (0.697 sec)
INFO:tensorflow:global_step/sec: 142.227
INFO:tensorflow:loss = 0.2840542509014079, step = 301 (0.704 sec)
INFO:tensorflow:global_step/sec: 142.356
INFO:tensorflow:loss = 0.3562266979420114, step = 401 (0.702 sec)
INFO:tensorflow:global_step/sec: 150.685
INFO:tensorflow:loss = 0.08145568292135648, step = 501 (0.665 sec)
INFO:tensorflow:global_step/sec: 150.351
INFO:tensorflow:loss = 0.05579175731337218, step = 601 (0.663 sec)
INFO:tensorflow:global_step/sec: 147.544
INFO:tensorflow:loss = 0.046771565348030585, step = 701 (0.678 sec)
INFO:tensorflow:global_step/sec: 163.539
INFO:tensorflow:loss = 0.04487715256866357, s

INFO:tensorflow:global_step/sec: 178.827
INFO:tensorflow:loss = 0.07171575384604541, step = 7501 (0.559 sec)
INFO:tensorflow:global_step/sec: 176.546
INFO:tensorflow:loss = 0.10024859949277488, step = 7601 (0.566 sec)
INFO:tensorflow:global_step/sec: 176.905
INFO:tensorflow:loss = 0.03334821547594518, step = 7701 (0.566 sec)
INFO:tensorflow:global_step/sec: 171.335
INFO:tensorflow:loss = 0.02941257196248247, step = 7801 (0.583 sec)
INFO:tensorflow:global_step/sec: 172.983
INFO:tensorflow:loss = 0.027875207445049736, step = 7901 (0.579 sec)
INFO:tensorflow:global_step/sec: 173.53
INFO:tensorflow:loss = 0.03836998349070495, step = 8001 (0.576 sec)
INFO:tensorflow:global_step/sec: 178.853
INFO:tensorflow:loss = 0.035413557832055555, step = 8101 (0.559 sec)
INFO:tensorflow:global_step/sec: 173.57
INFO:tensorflow:loss = 0.04325807428736684, step = 8201 (0.576 sec)
INFO:tensorflow:global_step/sec: 178.025
INFO:tensorflow:loss = 0.05923623616713722, step = 8301 (0.562 sec)
INFO:tensorflow:glo

INFO:tensorflow:global_step/sec: 175.875
INFO:tensorflow:loss = 0.033310285802104635, step = 15001 (0.569 sec)
INFO:tensorflow:global_step/sec: 176.122
INFO:tensorflow:loss = 0.03368908148947117, step = 15101 (0.568 sec)
INFO:tensorflow:global_step/sec: 171.563
INFO:tensorflow:loss = 0.03369916085630682, step = 15201 (0.583 sec)
INFO:tensorflow:global_step/sec: 179.08
INFO:tensorflow:loss = 0.042955658260343824, step = 15301 (0.558 sec)
INFO:tensorflow:global_step/sec: 176.043
INFO:tensorflow:loss = 0.05792990346151376, step = 15401 (0.568 sec)
INFO:tensorflow:global_step/sec: 174.451
INFO:tensorflow:loss = 0.07112827849950244, step = 15501 (0.573 sec)
INFO:tensorflow:global_step/sec: 172.535
INFO:tensorflow:loss = 0.12514224626458847, step = 15601 (0.580 sec)
INFO:tensorflow:global_step/sec: 175.561
INFO:tensorflow:loss = 0.12969740398235893, step = 15701 (0.570 sec)
INFO:tensorflow:global_step/sec: 173.689
INFO:tensorflow:loss = 0.04278293969261378, step = 15801 (0.576 sec)
INFO:tens

In [12]:
print (my_rmse_list)

[]
